## Tensorflow: gradient descent y tensorboard

In [43]:
import numpy as np
import pandas as pd
import tensorflow as tf
from datetime import datetime
from matplotlib import pyplot as plt


In [53]:
print(tf.__version__)

2.1.0


In [54]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Enabled compatitility to tf1.x


#### Carga de datos

In [7]:
data = np.load('proyecto_training_data.npy')

segmentacion de datos

In [209]:
largo = len(data)
cols = ['SalePrice','OverallQual','FstFlrSF','TotRmsAbvGrd','YearBuilt','LotFrontage']

tlargo = int(largo *1)
datatrain = np.nan_to_num(data[:tlargo])

dt = pd.DataFrame(datatrain,columns=cols)
dt.describe()

,SalePrice,OverallQual,FstFlrSF,TotRmsAbvGrd,YearBuilt,LotFrontage
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,180921.195890,6.099315,1162.626712,6.517808,1971.267808,57.623288
std,79442.502883,1.382997,386.587738,1.625393,30.202904,34.664304
min,34900.000000,1.000000,334.000000,2.000000,1872.000000,0.000000
25%,129975.000000,5.000000,882.000000,5.000000,1954.000000,42.000000
50%,163000.000000,6.000000,1087.000000,6.000000,1973.000000,63.000000
75%,214000.000000,7.000000,1391.250000,7.000000,2000.000000,79.000000
max,755000.000000,10.000000,4692.000000,14.000000,2010.000000,313.000000


#### Regularizacion de los datos

No es esta utlizando

In [46]:
x1_reg = dt[cols[1]]/120
y_reg = dt[cols[0]] /100000

### Clase con el calculo del gradiente

In [207]:
### Objeto del modelo
class GradientDesend:
    def __init__(self,b0=0.,b1=0.):
        tf.reset_default_graph()
        self.b0 = tf.get_variable("B0",dtype=tf.float32,
                                  shape=[],initializer=tf.constant_initializer(b0))
        self.b1 = tf.get_variable("B1",dtype=tf.float32,
                                  shape=[],initializer=tf.constant_initializer(b1))
        self.error = tf.get_variable("error",dtype=tf.float32,
                                  shape=[],initializer=tf.zeros_initializer())
        
    def step(self,x,y,lr):
        # hipotesis
        yhat = tf.matmul(x,[[self.b0],[self.b1]],name="yhat") 

        error = tf.reduce_mean(tf.math.square(y - yhat) ,name="reduce_mean")/2

        grad = tf.gradients(error,[self.b0,self.b1], name = "gradients")
        
        
        local_b0 = tf.assign(self.b0, self.b0 - lr*grad[0] , name="assign_b0")
        local_b1 = tf.assign(self.b1, self.b1 - lr*grad[1] , name="assign_b1")
        local_error = tf.assign(self.error, error , name="assign_error")
        
        
        salida = tf.group(local_b0,local_b1,local_error, name="grupo")

        return salida
        
        

In [205]:
#nc = np.ones_like(dt[cols[1]])
#x = np.hstack((np.expand_dims(dt[cols[1]],1),np.expand_dims(nc,1))) 
#x = dt[cols[1]]
#print(x)
#print(dt[cols[0]])

In [215]:
#plt.scatter(dt[cols[1]],dt[cols[0]])
tf.reset_default_graph()
init_b0 = 10
init_b1 = 1800
gd = GradientDesend(init_b0,init_b1)

lr = 0.05
iteracciones = 25

dat_x = tf.placeholder(tf.float32,[tlargo,2],"Value_x")
dat_y = tf.placeholder(tf.float32,[tlargo],"Value_y")

#prediccion = gd.calculate(dat_x)
step = gd.step(dat_x,dat_y,lr)


nc = np.ones_like(dt[cols[1]])
x = np.hstack((np.expand_dims(dt[cols[1]],1),np.expand_dims(nc,1))) 
#print(dat_x.shape)

#print(x.shape)
#print(dt[cols[0]].shape)
error_summary = tf.summary.scalar('error', gd.error)

with tf.train.MonitoredSession() as session:
    localdir = "logs/gradient_" + datetime.now().strftime("%Y%m%d-%H%M%S") + "_lr=" + str(lr) + "_b0="+ str(init_b0) + "_b1="+ str(init_b1)
    #print(localdir)
    feed_dict = {dat_x:x, dat_y:dt[cols[0]]}
    writer = tf.summary.FileWriter(localdir , session.graph)
    for i in range(iteracciones):
        print('------',i)
        
        local_med = session.run(step,feed_dict=feed_dict)
        #predicciones = session.run(prediccion,feed_dict=feed_dict)
        cb0,cb1,cerror = session.run([gd.b0,gd.b1,gd.error],feed_dict=feed_dict)
        summary = session.run(error_summary,feed_dict=feed_dict)
        print(cb0,cb1,cerror)
        #print(predicciones)
        #print(local_med)
        writer.add_summary(summary, i)
        # plt.plot(x,predicciones,label=str(i))

    
    
    # b0,b1,error = session.run([gd.b0,gd.b1,gd.error],feed_dict=feed_dict)
    #getattr(gd, "b1")
    #print(b0)
    #writer.add_summary(error, )
    
    #plt.title("Parámetros finales: m={}  b={} e={}".format(b0,b1,error))

#plt.legend()
#plt.show()

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
------ 0
54616.324 10753.025 19184670000.0
------ 1
-298.55078 2605.333 19281152000.0
------ 2
54665.51 11612.168 19378117000.0
------ 3
-607.6758 3406.5068 19475845000.0
------ 4
54716.633 12467.561 19574290000.0
------ 5
-917.39844 4203.533 19673426000.0
------ 6
54769.54 13319.188 19773243000.0
------ 7
-1227.6172 4996.454 19873740000.0
------ 8
54824.13 14167.074 19974881000.0
------ 9
-1538.3594 5785.2783 20076712000.0
------ 10
54880.652 15011.208 20179261000.0
------ 11
-1849.8867 6569.998 20282650000.0
------ 12
54938.938 15851.707 20386700000.0
------ 13
-2161.871 7350.6953 20491393000.0
------ 14
54999.156 16688.543 20596838000.0
------ 15
-2474.6445 8127.3184 20703169000.0
------ 16
55061.125 17521.71 20810109000.0
------ 17
-2787.9375 8899.927 20917799000.0
------ 18
55124.99 18351.227 21026193000.0
------ 19
-3102.0 9668.49 21135473000.0
------ 20
55190.6

In [40]:
localdir = "logs/gradient_" + datetime.now().strftime("%Y%m%d-%H%M%S") + "_"
localdir

'logs/gradient_20200519-215947_'

In [47]:
for i in range(5):
    print(i)

0
1
2
3
4


In [ ]:
## min 33:34
salida = tf.summary.scalar("cost",cost)

In [ ]:
writer = tf.summary.FileWriter('./logs/gradient_2020' + str(1), session.graph)

In [35]:
# definir el grafo
tf.reset_default_graph()
a = tf.constant(1.0)
b = tf.constant(1.0)
c = tf.constant(4.0)
d = tf.div(tf.add(a,b),c)


# executar el grafo
with tf.Session() as session:
    writer = tf.summary.FileWriter('./logs', session.graph)
    print(session.run([d,c])) 

[0.5, 4.0]


In [19]:
# definir el grafo
a = tf.constant(1.0)
b = tf.constant(1.0)
c = tf.constant(4.0)
d = tf.add(tf.add(a,b),c)

# executar el grafo
with tf.Session() as session:
  print(session.run(d)) 

6.0


In [172]:
tf.reset_default_graph()   # To clear the defined variables and operations of the previous cell

# create the scalar variable
x_scalar = tf.get_variable('x_scalar', shape=[], initializer=tf.truncated_normal_initializer(mean=0, stddev=1))

# ____step 1:____ create the scalar summary
first_summary = tf.summary.scalar('My_first_scalar_summary', x_scalar)

init = tf.global_variables_initializer()

# launch the graph in a session
with tf.Session() as sess:
    # ____step 2:____ creating the writer inside the session
    writer = tf.summary.FileWriter('./logs/test', sess.graph)
    for step in range(100):
        # loop over several initializations of the variable
        sess.run(init)
        # ____step 3:____ evaluate the scalar summary
        summary = sess.run(first_summary)
        # ____step 4:____ add the summary to the writer (i.e. to the event file)
        writer.add_summary(summary, step)
    print('Done with writing the scalar summary')
 


Done with writing the scalar summary


In [25]:
v = tf.get_variable(name="nombre",dtype=tf.float32,shape=[2,1], 
                    initializer=tf.random_normal_initializer(mean=0.0,stddev=0.5))

print(v)

<tf.Variable 'nombre:0' shape=(2, 1) dtype=float32_ref>


In [26]:
datos = tf.constant([[2.0,1.0],
                     [5.0,1.0]])
y = tf.matmul(datos,v)

print(y)

Tensor("MatMul:0", shape=(2, 1), dtype=float32)


In [28]:
x_eval = -4
def f(x):
  return tf.math.square(x)

x = tf.placeholder(tf.float32,[])
y = f(x)

derivada = tf.gradients(f(x),[x])

with tf.train.MonitoredSession() as session:
  feed_dict = {x:x_eval}

  valor_y,valor_derivada = session.run([y,derivada],feed_dict)
  print("derivada:",valor_derivada)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
derivada: [-8.0]


In [184]:
tf.reset_default_graph()
#localw = tf.get_variable(name="w",dtype=tf.float32,shape=[2,1],
#                    initializer=tf.random_normal_initializer(mean=0.0,stddev=0.5))
localw = tf.get_variable(name="w",dtype=tf.float32,#shape=[2,1],
                    initializer=tf.constant([[2.0],[1.0]]))

datos = tf.constant([[2.0,1.0],
                        [5.0,1.0],
                        [3,1]])
print(datos)
print(localw)
y = tf.matmul(datos,localw)
#with tf.train.MonitoredSession() as session:
#  print(session.run(y))

init = tf.global_variables_initializer() 
with tf.Session() as session:
  session.run(init)
  print(session.run(y))

Tensor("Const_1:0", shape=(3, 2), dtype=float32)
<tf.Variable 'w:0' shape=(2, 1) dtype=float32_ref>
[[ 5.]
 [11.]
 [ 7.]]


In [190]:
nc = np.ones_like(dt[cols[1]])
x = np.hstack((np.expand_dims(dt[cols[1]],1),np.expand_dims(nc,1))) 
#x = dt[cols[1]]
print(x)
print(dt[cols[0]])

dat_x = tf.placeholder(tf.float32,[tlargo,2],"Value_x")
dat_y = tf.placeholder(tf.float32,[tlargo,2],"Value_y")

print(dat_x)
#prediccion = gd.calculate(dat_x)
#modelo = gd.step(dat_x,dat_y,lr)


#nc = np.ones_like(dt[cols[1]])
#x = np.hstack((np.expand_dims(dt[cols[1]],1),np.expand_dims(nc,1))) 
#print(dat_x.shape)

#print(x.shape)
#print(dt[cols[0]].shape)

[[7. 1.]
 [6. 1.]
 [7. 1.]
 [7. 1.]
 [8. 1.]
 [5. 1.]
 [8. 1.]
 [7. 1.]
 [7. 1.]
 [5. 1.]
 [5. 1.]
 [9. 1.]
 [5. 1.]
 [7. 1.]]
0     208500.0
1     181500.0
2     223500.0
3     140000.0
4     250000.0
5     143000.0
6     307000.0
7     200000.0
8     129900.0
9     118000.0
10    129500.0
11    345000.0
12    144000.0
13    279500.0
Name: SalePrice, dtype: float64
Tensor("Value_x_5:0", shape=(14, 2), dtype=float32)
